检验
python -c "import tensorflow as tf; print('Num GPUs Available:', len(tf.config.list_physical_devices('GPU')))"

得先知道cudnn在哪
sudo find / -name cudnn_version.h 2>/dev/null
sudo find / -name cudnn_version.h 2>/dev/null | grep tfk

cat /home/xqgao/anaconda3/envs/tfk/lib/python3.11/site-packages/nvidia/cudnn/include/cudnn_version.h | grep CUDNN_MAJOR -A 2
#define CUDNN_MAJOR 9
#define CUDNN_MINOR 1
#define CUDNN_PATCHLEVEL 0
--
#define CUDNN_VERSION (CUDNN_MAJOR * 10000 + CUDNN_MINOR * 100 + CUDNN_PATCHLEVEL)
把他弄成860.
/home/xqgao/2025/SBU/cudnn-local-repo-ubuntu2004-8.9.6.50_1.0-1_amd64.deb

没关系，我们可以一步步查出来你现在的 cuDNN 是怎么装上的，然后再卸载它。  
Let's figure out how your current cuDNN was installed.

---

### ✅ Step 1: 检查是否通过 pip 安装（在 Conda 环境 `tfk` 中）

请运行以下命令：

```bash
pip list | grep cudnn
```

如果你看到如下类似输出：

```
nvidia-cudnn-cu11       9.1.0
```

就说明是用 `pip` 安装的。此时卸载方法是：

```bash
pip uninstall nvidia-cudnn-cu11
```

---

### ✅ Step 2: 检查是否通过 apt 安装

你可以这样检查：

```bash
dpkg -l | grep cudnn
```

如果你看到输出类似下面：

```
ii  libcudnn8  8.9.6.50-1+cuda11.8 ...
```

就说明是用 `.deb` 或 apt 安装的，此时卸载用：

```bash
sudo apt remove libcudnn8 libcudnn8-dev
```

---

### ✅ Step 3: 检查是否手动安装（tar 包复制进系统）

你已经发现这个文件：
```bash
/home/xqgao/anaconda3/envs/tfk/lib/python3.11/site-packages/nvidia/cudnn/include/cudnn_version.h
```

这说明 cuDNN 被某个包在这个目录下安装了。这种情况一般是 **pip 安装的 wheel 包**，比如 `nvidia-cudnn-cu11`，你可以直接删除这个目录：

```bash
rm -rf /home/xqgao/anaconda3/envs/tfk/lib/python3.11/site-packages/nvidia/cudnn
```

⚠️ 删除前可以先备份，以防误删：
```bash
mv /home/xqgao/anaconda3/envs/tfk/lib/python3.11/site-packages/nvidia/cudnn ~/cudnn_backup
```

---

### ✅ Step 4: 安装你想要的 cuDNN（v8.9.6）

你之前提供的 `.deb` 文件路径是：

```
/home/xqgao/2025/SBU/cudnn-local-repo-ubuntu2004-8.9.6.50_1.0-1_amd64.deb
```

可以按照下面完整流程操作：

```bash
cd /home/xqgao/2025/SBU
sudo dpkg -i cudnn-local-repo-ubuntu2004-8.9.6.50_1.0-1_amd64.deb
sudo cp /var/cudnn-local-repo-*/cudnn-local-*.key /usr/share/keyrings/
sudo apt update
sudo apt install libcudnn8=8.9.6.50-1+cuda11.8
sudo apt install libcudnn8-dev=8.9.6.50-1+cuda11.8
```

然后用这个命令确认：

```bash
cat /usr/include/cudnn_version.h | grep CUDNN_MAJOR -A 2
```

---

你可以先贴一下下面三个命令的输出，我来帮你判断现在的安装方式：
```bash
pip list | grep cudnn
dpkg -l | grep cudnn
ls /home/xqgao/anaconda3/envs/tfk/lib/python3.11/site-packages/nvidia/
```

🌟 如果你愿意，我也可以一步步带你做完。